# Kafka structured messages and avro encoding

A common use-case is to send messages payloads into kafka in JSON format

In [69]:
from kafka import KafkaProducer
import json

In [70]:
topic = "json"
server = "kafka:9092"
producer = KafkaProducer(bootstrap_servers=server, value_serializer=lambda v: json.dumps(v).encode('utf-8'))

First we need to load some data. Alas, the example apache log file is in unstructured common format.

In [71]:
src = "/home/jovyan/data/SDM/logs/apache-short.log"
with open(src, "r") as logfile:
    print(logfile.readline())

233.167.247.91 - - [26/Sep/2017:07:54:24 +0000] "POST /app/main/posts HTTP/1.0" 404 4948 "http://www.patel.org/post.asp" "Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_6_4) AppleWebKit/5360 (KHTML, like Gecko) Chrome/14.0.812.0 Safari/5360"



So I will need to parse this file into structured JSON. I will write a simple grok-ish parser for this purpose.

In [101]:
import re


grokpattern = "((?:\d{1,3}\.){3}\d{1,3}) - (\S+) \[(.+)\] \"(.+?)\" (\d+) (\d+) \"(.+?)\" \"(.+?)\""
grokpattern = re.compile(grokpattern)

grokData = []

with open(src, "r") as logfile:
    i = 0
    for line in logfile:
        i += 1
        vars = grokpattern.match(line)
        ip = vars.group(1)
        struct = {
            "ipv4": vars.group(1),
            "user": vars.group(2),
            "timestamp": vars.group(3),
            "request": vars.group(4),
            "response": int(vars.group(5)),
            "bytes": int(vars.group(6)),
            "referer": vars.group(7),
            "ua": vars.group(8)
        }
        grokData.append(struct)
        producer.send(topic, struct)

In [102]:
from kafka import KafkaConsumer
consumer = KafkaConsumer(topic, group_id=None, bootstrap_servers=server, auto_offset_reset='earliest')

i = 0
for msg in consumer:
    if i == 5:
        print(msg)
    i += 1
    if i == 10:
        break
consumer.close(autocommit = False)

ConsumerRecord(topic='json', partition=2, offset=5, timestamp=1519574845716, timestamp_type=0, key=None, value=b'{"ipv4": "233.167.247.91", "user": "-", "timestamp": "26/Sep/2017:07:54:24 +0000", "request": "POST /app/main/posts HTTP/1.0", "response": "404", "bytes": "4948", "referer": "http://www.patel.org/post.asp", "ua": "Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_6_4) AppleWebKit/5360 (KHTML, like Gecko) Chrome/14.0.812.0 Safari/5360"}', checksum=-914190552, serialized_key_size=-1, serialized_value_size=330)


Kafka stores raw bytes, it it's essentially agnostic to message source format. Storing such clearly sturctural data as bytearray from string is quite inefficient. For example, if we wanted to stora netflow, then kafka disk usage can fill quite fast. 

Avro is a popular bigdata encoding format that can be used to mitigate this issue. 

* https://avro.apache.org/docs/1.8.2/spec.html

Firstly, we will need to specify our message schema.

In [103]:
apacheSchema = {
    "name": "apache",
    "type": "record",
    "fields": [
        {
            "name": "ipv4",
            "type": "string"
        },
        {
            "name": "user",
            "type": "string"
        },
        {
            "name": "timestamp",
            "type": "string"
        },
        {
            "name": "request",
            "type": "string"
        },
        {
            "name": "response",
            "type": "int"
        },
        {
            "name": "bytes",
            "type": "int"
        },
        {
            "name": "referer",
            "type": "string"
        },
        {
            "name": "ua",
            "type": "string"
        }
    ]
}

In [104]:
import avro.schema, avro.io
import json
import io

In [105]:
schema = avro.schema.Parse(json.dumps(apacheSchema))

Initiate a new producer to another topic. Kafka will not stop you from sending mixed encoded and plain data into same topic, but it will make your like very difficult.

In [127]:
topic = "binary"
server = "kafka:9092"
producer = KafkaProducer(bootstrap_servers=server)

Encode data and send it to kafka.

In [128]:
for log in grokData:
    writer = avro.io.DatumWriter(schema)
    byteWriter = io.BytesIO()
    encoder = avro.io.BinaryEncoder(byteWriter)
    writer.write(log, encoder)
    raw = byteWriter.getvalue()
    producer.send(topic, raw)

In [134]:
consumer = KafkaConsumer(topic, group_id=None, bootstrap_servers=server, auto_offset_reset='earliest')

i = 0
for msg in consumer:
    if i % 10 == 0:
        print(msg)
    i += 1
    if i == 10:
        break
consumer.close(autocommit = False)

ConsumerRecord(topic='binary', partition=1, offset=0, timestamp=1519579033145, timestamp_type=0, key=None, value=b'\x1a44.19.216.236\x02-426/Sep/2017:07:54:24 +00008GET /app/main/posts HTTP/1.0\x90\x03\xf6N*http://www.wong.info/\xde\x01Mozilla/5.0 (Macintosh; PPC Mac OS X 10_8_0) AppleWebKit/5322 (KHTML, like Gecko) Chrome/13.0.805.0 Safari/5322', checksum=476458193, serialized_key_size=-1, serialized_value_size=211)


We now byte data when attempting to consume the topic. We are also missing JSON keys as our schema omits the need for those. To get the data out in raw format, we need to reverse the avro process.

In [139]:
reader = avro.io.DatumReader(schema)

In [141]:
consumer = KafkaConsumer(topic, group_id=None, bootstrap_servers=server, auto_offset_reset='earliest')

i = 0
for msg in consumer:
    i += 1
    bytes_reader = io.BytesIO(msg.value)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    payload = reader.read(decoder)
    if i == 10:
        print(payload)
        break

{'ipv4': '128.8.102.40', 'user': '-', 'timestamp': '26/Sep/2017:07:54:24 +0000', 'request': 'POST /explore HTTP/1.0', 'response': 200, 'bytes': 4961, 'referer': 'http://www.nichols-cook.com/index/', 'ua': 'Mozilla/5.0 (Windows CE; sl-SI; rv:1.9.0.20) Gecko/2011-06-06 21:48:50 Firefox/3.6.13'}
